抽象数据类型“栈”是一个有次序的数据集，每个数据项仅从“栈顶”一段加入到数据集中、从数据集中移除，栈具有后进先出LIFO的特性

栈有如下的操作：
- Stack() 创建一个空栈，不包含任何的数据项
- push(item) 将item加入栈顶，无返回值
- pop() 将栈顶数据项移除，并返回，栈被修改
- peek() 查看栈顶数据项，返回栈顶的数
- isEmpty() 返回栈是否为空栈
- size() 返回栈中有多少个数据项

栈在Python中可以用list来实现，list的两端都可以选择为栈的栈顶，但通常选择用list的末端，因为可以直接调用append和pop来实现push和pop功能，其性能为O(1)

In [2]:
#Stack类代码见stack.py
from stack import Stack

栈的应用：

1. 括号的匹配

In [3]:
def parChecker(symbolString):
    s = Stack()
    balanced = True #是否匹配
    index = 0
    while index < len(symbolString) and balanced: #遍历所有括号
        symbol = symbolString[index]
        if symbol == "(": #如果左括号，则入栈，等后续右括号匹配
            s.push(symbol)
        else: #如果右括号
            if s.isEmpty(): #看栈内是否有左括号
                balanced = False #没有则说明不匹配
            else:
                s.pop() #栈顶有左括号则弹出栈
        
        index = index + 1 #进行下一个符号判断

    if balanced and s.isEmpty():
        return True #循环后，栈内无多余左括号且匹配记录为真则说明匹配成功
    else:
        return False #不然为假

In [4]:
#测试
print(parChecker('((()))'))
print(parChecker('(()'))

True
False


In [5]:
#如果要进行通用符号的匹配判断，则可以在上面代码中进行修改
def parChecker(symbolString):
    s = Stack()
    balanced = True
    index = 0
    while index < len(symbolString) and balanced:
        symbol = symbolString[index]
        if symbol in "([{": #匹配符号的类型扩展为([{三种
            s.push(symbol)
        else: #如果右括号
            if s.isEmpty(): #看栈内是否有左括号
                balanced = False #没有则说明不匹配
            else:
                top = s.pop()
                if not matches(top, symbol): #左右符号的判断需要调用一个函数
                    balanced = False
        
        index = index + 1

    if balanced and s.isEmpty():
        return True
    else:
        return False
    
def matches(open, close): #开闭符判断函数
    opens = "([{"
    closers = ")]}"
    return opens.index(open) == closers.index(close)

In [6]:
#测试
print(parChecker('{{([][])}()}'))
print(parChecker('[{()]'))

True
False


栈的应用：

2. 十进制转换二进制

In [9]:
def divideBy2(decNumber):
    remstack = Stack() # 用来存放二进制的数

    while decNumber > 0:
        rem = decNumber % 2 # 求余数
        remstack.push(rem) #存放低位余数
        decNumber = decNumber // 2 #取整

    binString = ""
    while not remstack.isEmpty():
        binString = binString + str(remstack.pop()) # 把栈内的二进制结果拼接起来

    return binString

In [10]:
#测试
print(divideBy2(42))

101010


In [11]:
#扩展到更多进制计算，如八进制，十六进制
def baseConverter(decNumber, base):
    digits = "0123456789ABCDEF" #使用字符串来对应不同进制的字符表示

    remstack = Stack()

    while decNumber > 0:
        rem = decNumber % base #使用进制的基底来求余
        remstack.push(rem)
        decNumber = decNumber // base

    newString = ""
    while not remstack.isEmpty():
        newString = newString + digits[remstack.pop()]

    return newString

In [15]:
#测试
print(baseConverter(44,2))
print(baseConverter(44,16))

101100
2C


栈的应用：

3. 表达式转换
    - 中缀表达式，如：a + b * c，常规的数学表达式写法，对应全括号版本 (a + (b * c))，说明了计算的优先级
    - 前缀表达式，如：+ a * b c，运算符前置，运算符后面紧跟左右算子的优先计算，相当于在全括号中缀表达式中，将运算符放在对应左括号位置并消除对应右括号
    - 后缀表达式，如：a b c * +，运算符后置，运算符前面紧接左右算子的有限计算，相当于在全括号中缀表达式中，将运算符放在对应右括号位置并消除对应左括号

算法：

中缀表达式转后缀表达式算法（从左至右扫描中缀表达式单词列表）
- 如果单词是操作数，则直接添加到后缀表达式列表的词尾
- 如果单词是左括号“(”，则压入opstack栈顶
- 如果单词是右括号“)”，则反复弹出opstack栈顶操作符，加入到输出列表末尾，直到碰到左括号
- 如果单词是操作符“*/+-”，则压入opstack栈顶
    - 但在压入之前，要比较其与其他栈顶操作符的优先级
    - 如果栈顶的高于或等于它，就要反复弹出栈顶操作符，加入到输出列表的末尾
    - 直到栈顶的操作符优先级低于它
- 扫描结束后，把opstack栈中的所有剩余操作符一次弹出，添加到输出列表末尾
- 把输出列表用join方法合并成后缀表达式字符串

In [16]:
i = '3 * a + 5'
i.split()

['3', '*', 'a', '+', '5']

In [45]:
def infixToPostfix(infixexpr):
    prec = {} # 用字典保存运算符优先级
    prec["**"] = 4
    prec["*"] = 3
    prec["/"] = 3
    prec["+"] = 2
    prec ["-"] = 2
    prec["("] = 1
    opStack = Stack()
    postfixList = []
    tokenList = infixexpr.split() #把中缀表达式拆成逐个字符

    for token in tokenList:
        if token in "ABCDEFGHIJKLMNOPQRSTUVWXYZ" or token in "0123456789":
            postfixList.append(token)
        elif token == '(': # 如果是左括号，直接压入栈
            opStack.push(token)
        elif token == ')': # 如果是右括号，就从栈弹出，直到弹出其对应的左括号为止
            topToken = opStack.pop()
            while topToken != '(':
                postfixList.append(topToken)
                topToken = opStack.pop()
        else: #如果是运算符
            while (not opStack.isEmpty()) and (prec[opStack.peek()] >= prec[token]): #栈顶运算符优先级比扫描的运算符高的话
                postfixList.append(opStack.pop()) #弹出栈顶并加入列表
            opStack.push(token)

    while not opStack.isEmpty(): #扫描完成后，将栈反复弹出进列表
        postfixList.append(opStack.pop())
    return ' '.join(postfixList)

In [38]:
#测试
print(infixToPostfix("( A + B ) * ( C + D )"))
print(infixToPostfix("( A + B ) * C"))
print(infixToPostfix("A + B * C"))

A B + C D + *
A B + C *
A B C * +


栈的应用：

4. 后缀表达式的求值
    - 创建空栈operandStack用于暂存操作数
    - 将后缀表达式用split方式解析为单词列表
    - 从左到右扫描单词列表
        - 如果单词是一个操作数，将单词转换为整数int，压入operandStack栈顶
        - 如果单词是一个操作符（*/+-），就开始求值，从栈顶弹出2个操作数，先弹出的是右操作数，后弹出的是左操作数，计算后将值重新压入栈顶
    - 单词列表扫描结束后，表达式的值就在栈顶
    - 弹出栈顶的值，返回

In [43]:
def postfixEval(postfixExpr):
    operandStack = Stack()
    tokenList = postfixExpr.split()

    for token in tokenList:
        if token in "0123456789":
            operandStack.push(int(token))
        else:
            operand2 = operandStack.pop()
            operand1 = operandStack.pop()
            result  = doMath(token, operand1, operand2)
            operandStack.push(result)

    return operandStack.pop() #弹出结果

def doMath(op, op1, op2):
    if op == "*":
        return op1 * op2
    elif op == "**":
        return op1 ** op2
    elif op == "/":
        return op1 / op2
    elif op == "+":
        return op1 + op2
    else:
        return op1 - op2

In [40]:
#测试
postfixEval(infixToPostfix("( 3 * 5 ) + ( 4 * 6 )"))

39

In [47]:
infixToPostfix("5 * 3 ** ( 4 - 2 )")

'5 3 4 2 - ** *'